In [2]:
from statsmodels.regression.rolling import RollingOLS 
import  pandas as pd
import  numpy as np
import  matplotlib.pyplot as plt
import  statsmodels.api as sm
import  pandas_datareader.data as web
import  datetime as dt
import  yfinance as yf
import  pandas_ta
import  warnings
warnings.filterwarnings("ignore")

In [3]:
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
sp500[0]['Symbol'] = sp500[0]['Symbol'].str.replace('.', '-')
sp500_list = sp500[0]['Symbol'].unique().tolist()

end_date = '2023-01-22'
start_date = pd.to_datetime(end_date) - pd.DateOffset(365*8)
df = yf.download(tickers=sp500_list, start=start_date, end=end_date).stack()
print(df.head(5))

[*********************100%%**********************]  503 of 503 completed

2 Failed downloads:
['KVUE', 'VLTO']: Exception("%ticker%: Data doesn't exist for startDate = 1422075600, endDate = 1674363600")


Price              Adj Close      Close       High        Low       Open  \
Date       Ticker                                                          
2015-01-26 A       36.304653  39.150002  39.169998  38.470001  38.790001   
           AAL     52.539078  55.450001  56.200001  54.730000  55.889999   
           AAPL    25.310116  28.275000  28.590000  28.200001  28.434999   
           ABBV    42.971264  62.830002  63.040001  61.680000  62.259998   
           ABT     37.234196  44.150002  44.200001  43.419998  43.740002   

Price                   Volume  
Date       Ticker               
2015-01-26 A         1510900.0  
           AAL      13353900.0  
           AAPL    222460000.0  
           ABBV      8181100.0  
           ABT       4376200.0  


In [4]:
df.columns = ['adj close', 'close', 'high', 'low', 'open', 'volume']  
df.index.names = ['date', 'ticker']
df.head(5)

adj close      close       high        low       open  \
date       ticker                                                          
2015-01-26 A       36.304653  39.150002  39.169998  38.470001  38.790001   
           AAL     52.539078  55.450001  56.200001  54.730000  55.889999   
           AAPL    25.310116  28.275000  28.590000  28.200001  28.434999   
           ABBV    42.971264  62.830002  63.040001  61.680000  62.259998   
           ABT     37.234196  44.150002  44.200001  43.419998  43.740002   

                        volume  
date       ticker               
2015-01-26 A         1510900.0  
           AAL      13353900.0  
           AAPL    222460000.0  
           ABBV      8181100.0  
           ABT       4376200.0

In [11]:
df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open']))**2)

df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
                                                          
df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
                                                          
df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])


In [34]:
df['close'].dropna()

date        ticker
2015-01-26  A          39.150002
            AAL        55.450001
            AAPL       28.275000
            ABBV       62.830002
            ABT        44.150002
                         ...    
2023-01-20  YUM       126.629997
            ZBH       124.089996
            ZBRA      297.029999
            ZION       51.480000
            ZTS       163.809998
Name: close, Length: 989448, dtype: float64

In [45]:
def compute_atr(stock_data):
    atr = pandas_ta.atr(high=stock_data['high'],
                        low=stock_data['low'],
                        close=stock_data['close'],
                        length=14)
    return atr.sub(atr.mean()).div(atr.std())

df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

def compute_macd(close):
    macd_result = pandas_ta.macd(close=close, length=20)
    
    if macd_result is not None:
        macd = macd_result.iloc[:, 0]
        return macd.sub(macd.mean()).div(macd.std())
    else:
        # Handle the case where macd_result is None
        return None

df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)

mean_volume = np.mean(df['volume'])
df['dollar_volume'] = (df['close'] * df['volume'] / mean_volume).astype(int)


In [46]:
df['dollar_volume']

date        ticker
2015-01-26  A           11
            AAL        143
            AAPL      1220
            ABBV        99
            ABT         37
                      ... 
2023-01-20  YUM         40
            ZBH         41
            ZBRA        17
            ZION        24
            ZTS        154
Name: dollar_volume, Length: 989448, dtype: int32